# Cloud Data Transfer Speeds Benchmarking Workflow

Add overview of workflow

## Step 0: Load Required Setup Packages & Classes

Installs required workflow setup packages and calls UI generation script. If one or more of the packages don't exist in your `base` environment, they will install for you. Note that if installation is required, this cell will take a few minutes to complete execution.

In [ ]:
import os
import json
import sys
os.system("bash " + os.getcwd() + "/utils/jupyter-user-input/install-ui-packages.sh")
sys.path.insert(0, os.getcwd() + '/utils/jupyter-user-input')
import ui_helpers as ui

## Step 1: Define Workflow Inputs

### Cloud Compute Resources

In [ ]:
resource = ui.resourceWidgets()
resource.display()
resources = resource.processInput()
resources

### Cloud Object Stores

In [ ]:
store = ui.storageWidgets()
store.display()
storage = store.processInput()
storage

### Randomly Generated File Options

In [ ]:
randgen = ui.randgenWidgets(resources=resources)
randgen.display()
randfiles = randgen.processInput()
randfiles

## Step 2: Notebook Setup

In [ ]:
print('Setting up workflow...')

user_input = json.dumps({"RESOURCES" : resources,
             "STORAGE" : storage,
             "RANDFILES" : randfiles})

with open('user_input.json', 'w') as outfile:
    outfile.write(user_input)


#! chmod u+x workflow_notebook_setup.sh
#! ./workflow_notebook_setup.sh

print('Workflow setup complete.')